# Aнализ взаимодействия пользователей с карточками Яндекс.Дзен

In [1]:
!pip install pandas -U
!pip install sqlalchemy -U
!pip install psycopg2 -U

In [2]:
# импортируем библиотеки
import pandas as pd
from sqlalchemy import create_engine

**Создадим коннекцию к базе данных**

In [3]:
db_config = {'user': 'praktikum_student', # имя пользователя
            'pwd': 'Sdf4$2;d-d30pp', # пароль
            'host': 'rc1b-wcoijxj3yxfsf3fs.mdb.yandexcloud.net',
            'port': 6432, # порт подключения
            'db': 'data-analyst-zen-project-db'} # название базы данных

connection_string = 'postgresql://{}:{}@{}:{}/{}'.format(db_config['user'],
                                                db_config['pwd'],
                                                db_config['host'],
                                                db_config['port'],
                                                db_config['db'])

engine = create_engine(connection_string) 

In [4]:
# получаем сырые данные
query = ''' SELECT * FROM dash_visits
        '''

In [7]:
dash_visits = pd.io.sql.read_sql(query, con = engine) 

In [8]:
dash_visits.head()

,record_id,item_topic,source_topic,age_segment,dt,visits
0,1040597,Деньги,Авто,18-25,2019-09-24 18:32:00,3
1,1040598,Деньги,Авто,18-25,2019-09-24 18:35:00,1
2,1040599,Деньги,Авто,18-25,2019-09-24 18:54:00,4
3,1040600,Деньги,Авто,18-25,2019-09-24 18:55:00,17
4,1040601,Деньги,Авто,18-25,2019-09-24 18:56:00,27


**Посмотрим на данные**

In [9]:
def info_about_df(row):
    
    print(f'Количество строк - {row.shape[0]}')
    print(f'Количество столбцов - {row.shape[1]}')    
    print()
    print()
    
    print('\033[1m'+'Посмотрим на 5 первых строк датафрейма:'+'\033[0m')    
    display(row.head())
    print()
    print()

    print('\033[1m'+'Посмотрим на описание числовых данных:'+'\033[0m')     
    display(row.describe())
    print()
    print()
    
    print('\033[1m'+'Количество уникальных значений:'+'\033[0m')
    for i in row.columns:
        print('Колонка - ',i)
        print('    Уникальных значений - ',row[i].nunique())
        print()
    print()
        
    print('\033[1m'+'Посмотрим на типы данных:'+'\033[0m')  
    print(row.info())
    print()
    print()
        
    print('\033[1m'+'Количество пропусков в колонках:'+'\033[0m')
    print(row.isna().sum())
    print('\033[1m'+'Посмотрим на пропуски в процентах:'+'\033[0m')
    print(round((row.isna().mean()*100),2))
    print()
    print()

    print('\033[1m'+'Посмотрим на количество дубликатов по всем колонкам:'+'\033[0m')    
    print(row.duplicated().sum())      
    print(f'В процентах от количества всех строк - {round(row.duplicated().sum()/row.shape[0]*100,2)}%')

In [10]:
info_about_df(dash_visits)

Количество строк - 30745
Количество столбцов - 6


Посмотрим на 5 первых строк датафрейма:


,record_id,item_topic,source_topic,age_segment,dt,visits
0,1040597,Деньги,Авто,18-25,2019-09-24 18:32:00,3
1,1040598,Деньги,Авто,18-25,2019-09-24 18:35:00,1
2,1040599,Деньги,Авто,18-25,2019-09-24 18:54:00,4
3,1040600,Деньги,Авто,18-25,2019-09-24 18:55:00,17
4,1040601,Деньги,Авто,18-25,2019-09-24 18:56:00,27




Посмотрим на описание числовых данных:


,record_id,visits
count,3.074500e+04,30745.000000
mean,1.055969e+06,10.089673
std,8.875461e+03,19.727601
min,1.040597e+06,1.000000
25%,1.048283e+06,1.000000
50%,1.055969e+06,3.000000
75%,1.063655e+06,10.000000
max,1.071341e+06,371.000000




Количество уникальных значений:
Колонка -  record_id
    Уникальных значений -  30745

Колонка -  item_topic
    Уникальных значений -  25

Колонка -  source_topic
    Уникальных значений -  26

Колонка -  age_segment
    Уникальных значений -  6

Колонка -  dt
    Уникальных значений -  17

Колонка -  visits
    Уникальных значений -  212


Посмотрим на типы данных:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 30745 entries, 0 to 30744
Data columns (total 6 columns):
 #   Column        Non-Null Count  Dtype         
---  ------        --------------  -----         
 0   record_id     30745 non-null  int64         
 1   item_topic    30745 non-null  object        
 2   source_topic  30745 non-null  object        
 3   age_segment   30745 non-null  object        
 4   dt            30745 non-null  datetime64[ns]
 5   visits        30745 non-null  int64         
dtypes: datetime64[ns](1), int64(2), object(3)
memory usage: 1.4+ MB
None


Количество пропусков в колонках:
record_id  

**Изменим тип колонки record_id на object**

In [11]:
dash_visits['record_id'] = dash_visits['record_id'].astype('object') 

In [12]:
dash_visits.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 30745 entries, 0 to 30744
Data columns (total 6 columns):
 #   Column        Non-Null Count  Dtype         
---  ------        --------------  -----         
 0   record_id     30745 non-null  object        
 1   item_topic    30745 non-null  object        
 2   source_topic  30745 non-null  object        
 3   age_segment   30745 non-null  object        
 4   dt            30745 non-null  datetime64[ns]
 5   visits        30745 non-null  int64         
dtypes: datetime64[ns](1), int64(1), object(4)
memory usage: 1.4+ MB


**Сохраним данные в csv формате**

In [13]:
dash_visits.to_csv('dash_visits.csv', index = False)

## Теперь можно перейти к построению Дашборда